In [5]:
import importlib
import os, pandas as pd, numpy as np
from joblib import dump, load
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
# from sod.core.evaluation import normalize
import sod.core.dataset as sod_core_dataset
importlib.reload(sod_core_dataset)
import sod.core.plot as sod_plot
importlib.reload(sod_plot)
# from sod.core.dataset import open_dataset # , normalize_df
# from sod import plot
isoutl = sod_core_dataset.is_outlier

dataframe1 = sod_core_dataset.open_dataset('oneminutewindows.hdf', normalize=False, verbose=False)
dataframe2 = sod_core_dataset.open_dataset('magnitudeenergy.hdf', normalize=False, verbose=False)

print([_ for _ in dataframe1.columns if _ not in dataframe2.columns])
print([_ for _ in dataframe2.columns if _ not in dataframe1.columns])


# clf_iso_cont_5perc = load(os.path.abspath(os.path.join(os.getcwd(), '..', 'evaluations', 'results', 'cv.oneminutewindows.iforest.yaml',
#                          'IsolationForest?features=psd@2sec,psd@5sec&contamination=0.005&max_samples=16384&n_estimators=100&behaviour=new.model')))





# Copriamo tutti i casi: frequenze basse e frequenze medio alte (locali)
# psd0.2 psd5

['oneminutewindows.id', 'modified', 'window_type', 'start_time', 'length_sec']
['magnitudeenergy.id', 'subclass']


In [15]:
importlib.reload(sod_core_dataset)
from sod.core.dataset import magnitudeenergy, oneminutewindows, is_outlier, globalset
from sod.core import pdconcat

df1 = dataframe1.copy()
df2 = dataframe2.copy()

s2sid = 'Segment.db.id'
train_dfs = []
tests_dfs = []

df1.rename(columns={'oneminutewindows.id': s2sid, 'modified': globalset._SUBCLASS_COL}, inplace=True)
df1.insert(1, 'dataset_id', 1)
df1['dataset_id'] = df1['dataset_id'].astype(int) 
df1.drop(['start_time', 'length_sec'], 1, inplace=True)
# take from dataframe1 only noise and signals:
tmp_df1 = df1[~df1.window_type.str.match('^ns$')]
assert 3*len(tmp_df1)/2 == len(df1)
df1 = tmp_df1

df2.rename(columns={'magnitudeenergy.id': s2sid} , inplace=True)
df2.insert(1, 'dataset_id', 2)
df2['dataset_id'] = df2['dataset_id'].astype(int)
assert not df2.loc[magnitudeenergy.class_selector['unlabeled (unknown)'], 'outlier'].any()
assert df2.loc[magnitudeenergy.class_selector['unlabeled (suspicious outl.)'], 'outlier'].all()
df2[globalset._WINDOW_TYPE_COL] = ''

# build train: from dataframe1: inlier, and wrong inventory
globaldf = pdconcat([df1, df2])
expected = False
for cname in globalset.classnames:
    print(cname)
    assert (globaldf.loc[globalset.class_selector[cname], 'outlier'] == expected).all()
    expected = True

assert len(globaldf) > len(df1) and len(globaldf) > len(df2)

print('\nAll columns (same for both datasets):')
print (globaldf.columns)


ok
outl. (wrong inv)
outl. (cha. resp. acc <-> vel)
outl. (gain X100 or X0.01)
outl. (gain X10 or X0.1)
outl. (gain X2 or X0.5)
unlabeled (Me suspicious outl.)

All columns (same for both datasets):
Index(['Segment.db.id', 'dataset_id', 'psd@0.05sec', 'psd@0.1sec',
       'psd@0.2sec', 'psd@0.5sec', 'psd@1sec', 'psd@2sec', 'psd@3sec',
       'psd@5sec', 'psd@9sec', 'outlier', 'subclass', 'window_type',
       'amplitude_ratio', 'event_id', 'station_id', 'event_time',
       'channel_code', 'magnitude', 'distance_km'],
      dtype='object')


In [16]:
import pandas as pd
from sod.core.evaluation import save_df

save_df(globaldf, '/Users/riccardo/work/gfz/projects/sources/python/sod/sod/datasets/globalset.hdf', key='globalset')